In [1]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Subset
import numpy as np
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

/home/jaeyeob/anaconda3/envs/lstm/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.image import extract_patches_2d

digits = load_digits()
X, y = digits.images, digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

train_mask = np.isin(y_train, [1, 7])
X_train, y_train = X_train[train_mask], y_train[train_mask]

test_mask = np.isin(y_test, [1, 7])
X_test, y_test = X_test[test_mask], y_test[test_mask]

#
# X_train = X_train.reshape(X_train.shape[0], 16, 4)
# X_test = X_test.reshape(X_test.shape[0], 16, 4)

In [3]:
# 2x2 크기로 겹치지 않게 패치로 나누는 함수
def split_into_non_overlapping_patches(image, patch_size=(4, 4)):
    patches = []
    for i in range(0, image.shape[0], patch_size[0]):
        for j in range(0, image.shape[1], patch_size[1]):
            patch = image[i:i+patch_size[0], j:j+patch_size[1]].flatten()
            patches.append(patch)
    return np.array(patches)

In [4]:
# 각 이미지를 2x2 겹치지 않는 패치로 나누기
X_train = np.array([split_into_non_overlapping_patches(img) for img in X_train])
X_test = np.array([split_into_non_overlapping_patches(img) for img in X_test])

In [5]:
# y_train이 numpy 배열일 경우
unique, counts = np.unique(y_train, return_counts=True)
class_distribution = dict(zip(unique, counts))
print(class_distribution)

{1: 145, 7: 138}


In [6]:
# y_train이 numpy 배열일 경우
unique, counts = np.unique(y_test, return_counts=True)
class_distribution = dict(zip(unique, counts))
print(class_distribution)

{1: 37, 7: 41}


In [7]:
X_train.shape

(283, 4, 16)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Self-Attention and Binary Classifier from previous code
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert self.head_dim * heads == embed_size, "Embedding size must be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask=None):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split embedding into multiple heads
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out

# Classifier model
class SimpleSelfAttentionClassifier(nn.Module):
    def __init__(self, embed_size=16, heads=2, num_classes=2):
        super(SimpleSelfAttentionClassifier, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.fc = nn.Linear(embed_size * 4, num_classes)  # 49 is the sequence length in X_train

    def forward(self, x):
        attention_output = self.attention(x, x, x)
        attention_output = attention_output.flatten(start_dim=1)  # Flatten to feed into classifier
        out = self.fc(attention_output)
        return out

In [9]:
# Model, loss, optimizer
model = SimpleSelfAttentionClassifier(embed_size=16, heads=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
# def binary_accuracy(preds, y):
#     """
#     Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
#     """

#     #round predictions to the closest integer
#     rounded_preds = (torch.round(torch.sign(preds-0.5))+1)//2
#     correct = (rounded_preds == y).float() #convert into float for division 
#     acc = correct.sum() / len(correct)
#     return acc

In [11]:
# Define your binary accuracy function
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    
    return acc

In [13]:
# Training the model
for iepoch in tqdm(range(20)):
    optimizer.zero_grad()
    
    # Convert your X_train into tensor
    X_tensor = torch.tensor(X_train, dtype=torch.float32)
    
    # Forward pass
    predictions = model(X_tensor)
    
    # Convert y_train to tensor and change labels from 1 to 0, and 7 to 1
    label = torch.tensor(y_train, dtype=torch.long).clone()  # Ensure labels are LongTensor
    for i in range(len(label)):
        label[i] = 0 if label[i] == 1 else 1  # 1 -> 0, 7 -> 1
    
    # Compute loss
    loss = criterion(predictions, label)
    
    # Calculate binary accuracy (adjusted for your binary classification)
    acc = binary_accuracy(predictions.argmax(dim=1), label)
    
    # Print accuracy and loss
    print(f'Accuracy: {acc}')
    print(f'Loss: {loss.item()}')
    
    # Backward pass and optimization step
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {iepoch+1} complete")


  0%|          | 0/20 [00:00<?, ?it/s]

Accuracy: 0.4452296793460846
Loss: 1.6707100868225098


  5%|▌         | 1/20 [00:01<00:35,  1.86s/it]

Epoch 1 complete
Accuracy: 0.47703179717063904
Loss: 1.4804553985595703


 10%|█         | 2/20 [00:03<00:27,  1.54s/it]

...

Epoch 18 complete
Accuracy: 0.7208480834960938
Loss: 0.5712011456489563


 95%|█████████▌| 19/20 [00:34<00:02,  2.27s/it]

Epoch 19 complete
Accuracy: 0.7420494556427002
Loss: 0.5452451705932617


100%|██████████| 20/20 [00:36<00:00,  1.81s/it]

Epoch 20 complete


In [14]:
import torch
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, average_precision_score

# Convert test data to tensor
X_tensor = torch.tensor(X_test, dtype=torch.float32)
predictions = model(X_tensor)

# Convert y_train to tensor and change labels from 1 to 0, and 7 to 1
label = torch.tensor(y_test, dtype=torch.long).clone()  # Ensure labels are LongTensor
        
# Reverse the labels (if needed)
for i in range(len(label)):
    if label[i] == 1:
        label[i] = 0
    else:
        label[i] = 1

# Calculate loss and accuracy
# Compute loss
loss = criterion(predictions, label)

# Calculate binary accuracy (adjusted for your binary classification)
acc = binary_accuracy(predictions.argmax(dim=1), label)

# Convert predictions to probabilities using softmax
probabilities = torch.softmax(predictions, dim=1)  # Shape: (batch_size, 2)

# Extract probabilities for class 1 (positive class)
positive_class_probs = probabilities[:, 1].detach().numpy()

# Convert labels to numpy
labels_np = label.numpy()

# Calculate AUROC
auroc = roc_auc_score(labels_np, positive_class_probs)

# Binarize predictions for precision, recall, and F1 calculation
binary_preds = np.where(positive_class_probs > 0.5, 1, 0)

# Calculate Precision, Recall, and F1 Score
precision, recall, f1, _ = precision_recall_fscore_support(labels_np, binary_preds, average='binary')

# Calculate AUPRC (Area Under the Precision-Recall Curve)
auprc = average_precision_score(labels_np, positive_class_probs)

# Print results
print(f'\nAccuracy: {acc}\n')
print(f'Loss: {loss}\n')
print(f'AUROC: {auroc}\n')
print(f'Precision: {precision}\n')
print(f'Recall: {recall}\n')
print(f'F1 Score: {f1}\n')
print(f'AUPRC: {auprc}\n')


Accuracy: 0.7692307829856873

Loss: 0.5283698439598083

AUROC: 0.8226763348714569

Precision: 0.7948717948717948

Recall: 0.7560975609756098

F1 Score: 0.775

AUPRC: 0.7570751154003538

